In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (145, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
631834,350,350,265,144.238281,204.394531,225.0,11.621094,11.621094,5.0,-176.837097,-174.658203,-382.900146,0.683594,0.683594,5.0,1
697378,350,350,230,182.177734,218.750000,150.0,6.835938,15.722656,5.0,-177.563400,-174.658203,4.291016,0.683594,0.683594,5.0,31
696191,293,293,240,223.183594,153.367187,102.0,7.439453,8.583984,10.0,-142.213867,-257.213867,-172.500000,0.572266,0.572266,5.0,1
656321,312,312,285,176.414062,166.054688,245.0,6.093750,9.750000,5.0,-156.421646,-155.695312,-336.400635,0.609375,0.609375,5.0,31
388757,309,309,280,158.422852,129.755859,172.0,6.035156,12.673828,10.0,-141.198242,-300.198242,-883.500000,0.603516,0.603516,5.0,31
679995,329,329,265,94.137695,138.154297,150.0,5.140625,3.212891,5.0,-156.178711,-304.178711,-648.299988,0.642578,0.642578,5.0,1
656941,327,327,250,266.006836,194.475586,68.0,26.824219,19.160156,20.0,-165.180664,-303.180664,-477.200012,0.638672,0.638672,5.0,5
647585,403,403,270,227.868164,203.074219,173.0,14.167969,18.103516,10.0,-204.106445,-341.106445,-628.200012,0.787109,0.787109,5.0,31
509323,342,342,305,223.769531,127.582031,130.0,11.355469,8.683594,5.0,-156.866409,-170.666016,-491.956665,0.667969,0.667969,5.0,31


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_test.index))
print('distinct lungs in testset:', len(lungs))

distinct lungs in testset: 145


In [6]:
list_tasks = []
size = settings.CUBE_SIZE
step = 32
start = [ size//2, size//2, size//2 ] # z,y,x

for uid in tqdm(lungs):
    meta = df_meta_test.loc[uid] # slice
    
    for z in range(start[0], int(meta.slice), step):
        for y in range(start[1], int(meta.height), step):
            for x in range(start[2], int(meta.width), step):
                task = {}
                task['seriesuid'] = uid
                task['vcoordX'] = x
                task['vcoordY'] = y
                task['vcoordZ'] = z
                
                # print(task)
                list_tasks.append(task)

100%|██████████| 145/145 [00:00<00:00, 347.84it/s]


In [7]:
df_tasks = pd.DataFrame(list_tasks, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
df_tasks = df_tasks.set_index('seriesuid')
df_tasks.index = df_tasks.index.astype('str')

print('total:', df_tasks.shape, 'lung:', len(set(df_tasks.index)))

if not os.path.exists(settings.SUBMISSION_DIR + 'candidates'):
    os.mkdir(settings.SUBMISSION_DIR + 'candidates')
    
df_tasks.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_cubes.csv', encoding='utf-8')


total: (142434, 3) lung: 145


# 3. Predict Results by tasks_cubes.csv

In [8]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    SMOOTH = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + SMOOTH) / (K.sum(y_true_f) + K.sum(y_pred_f) + SMOOTH)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

output_dir = './output/unet3d/1564070613/' # labels in center coord
model = load_model(output_dir + "model-best.hd5", 
                   custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

W0728 23:08:41.985121 140312650372928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 23:08:41.987280 140312650372928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0728 23:08:41.994310 140312650372928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [10]:
batch_size = 32
threshold_probability = 0.8
wtype = 'medi'
plot = False

In [11]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [12]:
list_results = []
print(wtype, 'window')
for uid in set(df_tasks.index):
    tasks = df_tasks.loc[[uid]] # DataFrame
    meta = df_meta_test.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks), 'meta:', meta.slice, meta.height, meta.width)
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='test', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=settings.CUBE_SIZE)

        if np.sum(cube) > 0:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            print(X_test.shape)
            predictions_test = model.predict(X_test, batch_size=batch_size, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                pred = predictions_test[i] > threshold_probability
                if np.sum(pred) > 10:
                    vcoords, diameters, bboxes = helper.get_regions_detail(pred[:,:,:,0], itm)
                    if len(bboxes) > 0:
                        for i, box in enumerate(bboxes):
                            result = {}
                            result['seriesuid'] = uid
                            result['vcoordX'] = vcoords[i][2]
                            result['vcoordY'] = vcoords[i][1]
                            result['vcoordZ'] = vcoords[i][0]
                            result['diametersX'] = diameters[i][2]
                            result['diametersY'] = diameters[i][1]
                            result['diametersZ'] = diameters[i][0]
                            if plot:
                                plot_cube((pred*255).astype(np.uint8))

                            list_results.append(result)
                        
        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','vcoordX','vcoordY','vcoordZ','diametersX','diametersY','diametersZ'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)))

    df_results.to_csv(settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 688554 tasks: 1296 meta: 305.0 391.0 391.0
(1218, 64, 64, 64, 1)
1218/1218 [==============================] - 36s 29ms/sample
total: (196, 6) lung: 1
----------------------------------------------------------------------------------------------------
lung: 663949 tasks: 600 meta: 216.0 330.0 330.0
(569, 64, 64, 64, 1)
569/569 [==============================] - 18s 32ms/sample
total: (310, 6) lung: 2
----------------------------------------------------------------------------------------------------
lung: 687117 tasks: 1000 meta: 325.0 327.0 327.0
(993, 64, 64, 64, 1)
993/993 [==============================] - 28s 29ms/sample
total: (429, 6) lung: 3
----------------------------------------------------------------------------------------------------
lung: 383745 tasks: 384 meta: 220.0 276.0 276.0
(384, 64, 64, 64, 1)
384/384 [==============================] - 10s 26ms/sample
total: (567, 6) lung: 4
------------------------------------------------------------------------

(646, 64, 64, 64, 1)
646/646 [==============================] - 15s 23ms/sample
total: (5049, 6) lung: 33
----------------------------------------------------------------------------------------------------
lung: 680069 tasks: 968 meta: 260.0 367.0 367.0
(936, 64, 64, 64, 1)
936/936 [==============================] - 24s 26ms/sample
total: (5149, 6) lung: 34
----------------------------------------------------------------------------------------------------
lung: 423221 tasks: 1210 meta: 324.0 380.0 380.0
(991, 64, 64, 64, 1)
991/991 [==============================] - 23s 24ms/sample
total: (5398, 6) lung: 35
----------------------------------------------------------------------------------------------------
lung: 696762 tasks: 900 meta: 315.0 346.0 346.0
(884, 64, 64, 64, 1)
884/884 [==============================] - 22s 25ms/sample
total: (5573, 6) lung: 36
----------------------------------------------------------------------------------------------------
lung: 406559 tasks: 900 met

949/949 [==============================] - 28s 29ms/sample
total: (9664, 6) lung: 65
----------------------------------------------------------------------------------------------------
lung: 543664 tasks: 1210 meta: 333.0 370.0 370.0
(982, 64, 64, 64, 1)
982/982 [==============================] - 25s 25ms/sample
total: (9915, 6) lung: 66
----------------------------------------------------------------------------------------------------
lung: 704051 tasks: 1859 meta: 354.0 429.0 429.0
(1678, 64, 64, 64, 1)
1678/1678 [==============================] - 45s 27ms/sample
total: (10178, 6) lung: 67
----------------------------------------------------------------------------------------------------
lung: 655527 tasks: 1089 meta: 294.0 366.0 366.0
(1005, 64, 64, 64, 1)
1005/1005 [==============================] - 30s 30ms/sample
total: (10342, 6) lung: 68
----------------------------------------------------------------------------------------------------
lung: 655508 tasks: 700 meta: 230.0 32

(743, 64, 64, 64, 1)
743/743 [==============================] - 11s 14ms/sample
total: (14763, 6) lung: 97
----------------------------------------------------------------------------------------------------
lung: 657263 tasks: 1210 meta: 345.0 367.0 367.0
(1037, 64, 64, 64, 1)
1037/1037 [==============================] - 15s 15ms/sample
total: (14949, 6) lung: 98
----------------------------------------------------------------------------------------------------
lung: 686478 tasks: 567 meta: 245.0 307.0 307.0
(567, 64, 64, 64, 1)
567/567 [==============================] - 8s 14ms/sample
total: (15031, 6) lung: 99
----------------------------------------------------------------------------------------------------
lung: 512818 tasks: 1100 meta: 370.0 339.0 339.0
(898, 64, 64, 64, 1)
898/898 [==============================] - 17s 19ms/sample
total: (15213, 6) lung: 100
----------------------------------------------------------------------------------------------------
lung: 646230 tasks:

(782, 64, 64, 64, 1)
782/782 [==============================] - 15s 20ms/sample
total: (20196, 6) lung: 129
----------------------------------------------------------------------------------------------------
lung: 680250 tasks: 1584 meta: 360.0 403.0 403.0
(1363, 64, 64, 64, 1)
1363/1363 [==============================] - 24s 18ms/sample
total: (20472, 6) lung: 130
----------------------------------------------------------------------------------------------------
lung: 394788 tasks: 512 meta: 280.0 285.0 285.0
(512, 64, 64, 64, 1)
512/512 [==============================] - 9s 17ms/sample
total: (20585, 6) lung: 131
----------------------------------------------------------------------------------------------------
lung: 674155 tasks: 1440 meta: 350.0 386.0 386.0
(1387, 64, 64, 64, 1)
1387/1387 [==============================] - 26s 18ms/sample
total: (20823, 6) lung: 132
----------------------------------------------------------------------------------------------------
lung: 656185 

In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'candidates' + '/tasks_' + wtype +'.csv', 
                            index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))



results: (22662, 6) distinct lung: 145
